# Description

This notebooks guides you through the procedures to use DeepStream for RTSP Streaming

## 1. Source configurations

### 1.1 RTSP Source

Expected source0 group from DeepStream application configuration file:
- set type=4

- set uri=rtsp://\<ip>:\<host>/path      


### 1.2 4K Videos in the container

You may use your own videos download 4K videos from YouTube

In this demo, we prepared 3 4K videos
- https://www.youtube.com/watch?v=gYO1uk7vIcc
- https://www.youtube.com/watch?v=V8nBZbbJePs
- https://www.youtube.com/watch?v=eeBnpgWH2vQ

In [ ]:
!ls -l ../../video-src

If you wish to download by yourself, we check the formats for YouTube videos:

In [ ]:
!youtube-dl -F https://www.youtube.com/watch?v=V8nBZbbJePs

We will download vp9 formatted 4K video as av01 formatted video is not supported on DeepStream.

Modify -f flag and YouTube link with reference to previous cell output

In [ ]:
!cd ../../video-src && \
    youtube-dl -f 313 -o hk_4k.webm https://www.youtube.com/watch?v=V8nBZbbJePs

Verify video source

In [ ]:
!ls -l ../../video-src

Expected source0 group from DeepStream application configuration file:
- set type=3

- set uri=../video-src/\<video_file>


### Writing the config file in notebook
Edit the below cell, run the cell after edit and save the file to **../../app-src/deepstream_app_rtsp_source.txt**

In [ ]:
!mkdir ../../app-src

In [ ]:
%%writefile ../../app-src/deepstream_app_rtsp_source.txt
################################################################################
# Copyright (c) 2020, NVIDIA CORPORATION. All rights reserved.
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.  IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
################################################################################

[application]
enable-perf-measurement=1
perf-measurement-interval-sec=1

[tiled-display]
enable=1
rows=1
columns=1
width=1280
height=720
gpu-id=0

[source0]
enable=1
#Type - 1=CameraV4L2 2=URI 3=MultiURI
type=3
num-sources=1
uri=file://../video-src/hk_4k.webm
gpu-id=0

[streammux]
gpu-id=0
batch-size=1
batched-push-timeout=40000
## Set muxer output width and height
width=3840
height=2160

[sink0]
enable=1
#Type - 1=FakeSink 2=EglSink 3=File
type=1
sync=1
source-id=0
gpu-id=0

[osd]
enable=0

[primary-gie]
enable=0

[sink1]
enable=0
type=3
#1=mp4 2=mkv
container=1
#1=h264 2=h265 3=mpeg4
codec=1
#encoder type 0=Hardware 1=Software
enc-type=0
sync=0
bitrate=2000000
#H264 Profile - 0=Baseline 2=Main 4=High
#H265 Profile - 0=Main 1=Main10
profile=0
output-file=out.mp4
source-id=0

[sink2]
enable=1
#Type - 1=FakeSink 2=EglSink 3=File 4=RTSPStreaming 5=Overlay
type=4
#1=h264 2=h265
codec=1
#encoder type 0=Hardware 1=Software
enc-type=0
sync=0
bitrate=8000000
#H264 Profile - 0=Baseline 2=Main 4=High
#H265 Profile - 0=Main 1=Main10
profile=4
# set below properties in case of RTSPStreaming
rtsp-port=8554
udp-port=5400

[tracker]
enable=0

[tests]
file-loop=0


## 2. Sink (Output) Configurations

Configure samples/configs/tlt_pretrained_models/deepstream_app_source1_peoplenet.txt

In [ ]:
cat ../../app-src/deepstream_app_rtsp_source.txt

### 2.1 Graphical Application

Default sink (set enable=1 under sink0 group)

### 2.2 Saved output file

To view DeepStream streaming via output file saved, under sink1 group,
- set enable=1 
- set output-file=path/to/your/out-video  e.g. output-file=out-video/out.mp4

### 2.3 RTSP

To view DeepStream streaming via RTSP, under sink2 group
- set enable=1 

### Writing the config file in notebook
- Run the below cell
- Edit the below cell
- After edit, replace first line (**# %load ../../app-src/deepstream_app_rtsp_source.txt**) with **%%writefile ../../app-src/deepstream_app_rtsp_source.txt**
- Run the cell after edit to save the file to **../../app-src/deepstream_app_rtsp_source.txt**

In [ ]:
%load ../../app-src/deepstream_app_rtsp_source.txt

## 3. Run DeepStream

To start DeepStream, run the below cell

In [ ]:
!cd ../../app-src && \
    deepstream-app -c deepstream_app_rtsp_source.txt

If you set RTSP as sink, 
- download vlc media player at https://www.videolan.org/vlc/
- open vlc player, press ctrl+N, set network url to your rtsp url set in sink config.
- click play

* NOTE: Hardware setting e.g. bandwidth, network speed could affect RTSP Streaming 

DeepStream Runtime commands: P - pause, R - resume, Q - quit

DeepStream Runtime Performance will be shown as PERF (FPS)

## Technical Support

### Q1: What if I see "No protocol specified" as DeepStream failed to run?

Answer: your display setting is incorrect. If you are running on remote GPU accelerator like V100, please set the type under sink0 to 1 (Fakesink)

Contact NVIDIA HK team for support.